# Problem Statement
The objective is to forecast daily sales for Rossmann stores using historical sales data.
We compare traditional time-series models (ARIMA / Prophet) with a machine learning model (XGBoost).


## Setup & Dataset Upload

In [ ]:
from google.colab import files
files.upload()

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error

## Load & Prepare Data

In [ ]:
df = pd.read_csv('train.csv')
df.head()

### Convert Date & sort

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

### Filter one store ( Forecasting is easier and clearer on one store.)

In [ ]:
store_df = df[df['Store'] == 1][['Date', 'Sales']]

## Train–Test Split

In [ ]:
train = store_df.iloc[:-90]
test = store_df.iloc[-90:]

# 🔵 MODEL 1: ARIMA

### ARIMA Model

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arima_model = ARIMA(train['Sales'], order=(5,1,0))
arima_fit = arima_model.fit()

arima_pred = arima_fit.forecast(steps=len(test))

 ### ARIMA Metrics

In [ ]:
mae_arima = mean_absolute_error(test['Sales'], arima_pred)
rmse_arima = np.sqrt(mean_squared_error(test['Sales'], arima_pred))

# 🟢 MODEL 2: Prophet

### Install & Prepare Prophet

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet

#### Prepare data

In [ ]:
prophet_train = train.rename(columns={'Date':'ds', 'Sales':'y'})

#### Train model

In [ ]:
prophet = Prophet()
prophet.fit(prophet_train)

#### Forecast

In [ ]:
future = prophet.make_future_dataframe(periods=90)
forecast = prophet.predict(future)

#### Extract predictions

In [ ]:
prophet_pred = forecast.iloc[-90:]['yhat']

### Prophet Metrics

In [ ]:
mae_prophet = mean_absolute_error(test['Sales'], prophet_pred)
rmse_prophet = np.sqrt(mean_squared_error(test['Sales'], prophet_pred))

## 🔴 MODEL 3: XGBoost (ML Approach)

### Feature Engineering

In [ ]:
store_df['day'] = store_df['Date'].dt.day
store_df['month'] = store_df['Date'].dt.month
store_df['year'] = store_df['Date'].dt.year
store_df['weekday'] = store_df['Date'].dt.weekday

#### Split again:

In [ ]:
X = store_df.drop(['Sales','Date'], axis=1)
y = store_df['Sales']

X_train, X_test = X.iloc[:-90], X.iloc[-90:]
y_train, y_test = y.iloc[:-90], y.iloc[-90:]

### Train XGBoost

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    random_state=42
)

xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

### XGBoost Metrics

In [ ]:
mae_xgb = mean_absolute_error(y_test, xgb_pred)
rmse_xgb = np.sqrt(mean_squared_error(y_test, xgb_pred))

## Model Comparison

In [ ]:
results = pd.DataFrame({
    'Model': ['ARIMA', 'Prophet', 'XGBoost'],
    'MAE': [mae_arima, mae_prophet, mae_xgb],
    'RMSE': [rmse_arima, rmse_prophet, rmse_xgb]
})

results

# Final Insights

## Model Comparison & Insights

- ARIMA captures short-term trends but struggles with complex patterns.
- Prophet handles seasonality better than ARIMA.
- XGBoost achieved the lowest error by leveraging engineered time-based features.
- Machine learning models outperform classical time-series models when sufficient features are available.